![head.png](https://github.com/iwh-halle/FinancialDataAnalytics/blob/master/figures/head.jpg?raw=1)

# Financial Data Analytics in Python

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
MLU - Martin Luther University Halle-Wittenberg

fabian.woebbeking@iwh-halle.de

# Homework

You will need a Git/GitHub repository to submit your course deliverables. Consult [**slides.ipynb**](https://github.com/iwh-halle/FinancialDataAnalytics) for help with the tasks below! If you need further assistance, do not hesitate to open a Q&A at https://github.com/cafawo/FinancialDataAnalytics/discussions

### Task: 

The liquidity position of a firm measured in million is a **generalized Wiener process** with a drift of $0.1$ per month and a variance of $\sigma^2 = 0.16$ per month. The initial cash position is $2.0$. Calculate:

1. 	the expected value and standard deviation in one, six and 12 months.
2.	What is the probability of a negative cash position in 6 and 12 months? 

In [2]:
import numpy as np


drift = 0.1  # drift per month
variance = 0.16  # variance per month
initial_cash = 2.0  # initial cash position in million dollars

# Time points (in months)
time_points = [1, 6, 12]

# Calculate expected value and standard deviation for each time point
for t in time_points:
    # Expected value
    expected_value = initial_cash + drift * t
    
    # Standard deviation
    standard_deviation = np.sqrt(variance * t)
    
    # Print results
    print(f"At {t} months:")
    print(f"Expected value: {expected_value:.2f} million dollars")
    print(f"Standard deviation: {standard_deviation:.2f} million dollars\n")
    
    
    #%%
    from scipy.stats import norm

# Function to calculate probability of negative cash position
def prob_negative_cash_position(initial_cash, drift, variance, t):
    # Calculate the mean and standard deviation of the cash position at time t
    mean = initial_cash + drift * t
    standard_deviation = np.sqrt(variance * t)
    
    # Use the cumulative distribution function (CDF) of the standard normal distribution
    # to find the probability of the cash position being negative
    prob_negative = norm.cdf(0, loc=mean, scale=standard_deviation)
    
    return prob_negative

# Calculate and print the probability of negative cash position at 6 and 12 months
prob_negative_6_months = prob_negative_cash_position(initial_cash, drift, variance, 6)
prob_negative_12_months = prob_negative_cash_position(initial_cash, drift, variance, 12)

print(f"Probability of a negative cash position in 6 months: {prob_negative_6_months:.4f}")
print(f"Probability of a negative cash position in 12 months: {prob_negative_12_months:.4f}") 

At 1 months:
Expected value: 2.10 million dollars
Standard deviation: 0.40 million dollars

At 6 months:
Expected value: 2.60 million dollars
Standard deviation: 0.98 million dollars

At 12 months:
Expected value: 3.20 million dollars
Standard deviation: 1.39 million dollars

Probability of a negative cash position in 6 months: 0.0040
Probability of a negative cash position in 12 months: 0.0105


### Task: 

The cash flow of a [call option](https://en.wikipedia.org/wiki/Call_option) with strike $K$ at maturity $T$ is given by

$$
max(S_T - K, 0) = (S_T - K)^+
$$

where $S_T$ is the price of the underlying at $T$. The price of the option under the [risk-neutral measure](https://en.wikipedia.org/wiki/Risk-neutral_measure) $\mathbb{Q}$ is simply its discounted expected value
$$
\mathbb{E}^\mathbb{Q}[(S_T - K)^+] e^{-rT}.
$$


Calculate the price of the option, using:
1. numerical integration and
2. Monte carlo simulation.

For you calculations, assume that todays price of the underlying is $S_0 = 220$, the strike is $K = 220$, volatility is $\sigma = 0.98$, the risk free rate is $r = 10\%$ (continuous) and maturity is one year. We further assume that the underlying $S$ follows a **Geometric Brownian motion**.

Hint: The terminal stock price $S_T$, under the risk-neutral measure, follows a log-normal distribution with PDF

$$f(x) = \frac{1}{x s \sqrt{2 \pi}} \exp\left( -\frac{(\ln x - \mu)^2}{2 s^2} \right) $$

where $\mu = \ln S_0 + (r-\sigma^2 / 2)T$ and variance $s^2 = \sigma^2 T$.


In [1]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

# Given parameters
S0 = 220  # today's price of the underlying
K = 220   # strike price
sigma = 0.98  # volatility
r = 0.10  # risk-free rate
T = 1  # time to maturity

# Calculating parameters for the log-normal distribution
mu = np.log(S0) + (r - 0.5 * sigma**2) * T
s = sigma * np.sqrt(T)

# 1. Numerical Integration
def integrand(x):
    return np.exp(-r * T) * max(x - K, 0) * (1 / (x * s * np.sqrt(2 * np.pi))) * np.exp(-((np.log(x) - mu)**2 / (2 * s**2)))

option_price_integration, _ = quad(integrand, K, np.inf)

# 2. Monte Carlo Simulation
np.random.seed(0)  # for reproducibility
n_simulations = 1000000
ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * np.random.randn(n_simulations))
payoffs = np.maximum(ST - K, 0)
option_price_monte_carlo = np.exp(-r * T) * np.mean(payoffs)

# Results
print(f"Option price using numerical integration: {option_price_integration:.2f}")
print(f"Option price using Monte Carlo simulation: {option_price_monte_carlo:.2f}")

Option price using numerical integration: 89.60
Option price using Monte Carlo simulation: 89.75
